In [ ]:
import keras
import tensorflow
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows,img_cols,channels)

In [ ]:
def build_generator():
  noise_shape = (100,)
  model = Sequential()
  
  model.add(Dense(256,input_shape=noise_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(512))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(1024))
  model.add(LeakyReLU(alpha=0.2))
  model.add(BatchNormalization(momentum=0.8))

  model.add(Dense(np.prod(img_shape),activation='tanh'))
  model.add(Reshape(img_shape))

  model.summary()

  noise = Input(shape=noise_shape)
  img = model(noise)

  return Model(noise,img)

In [ ]:
def build_discriminator():


    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [ ]:
def train(epochs,batch_size=128,save_interval=500):
  (X_train,_),(_,_) = mnist.load_data()
  X_train = (X_train.astype(np.float32)-127.5)/127.5
  X_train = np.expand_dims(X_train,axis=3)
  half_batch = int(batch_size/2)

  for epoch in range(epochs):
    idx = np.random.randint(0,X_train.shape[0],half_batch)
    imgs = X_train[idx]
    
    noise = np.random.normal(0,1,(half_batch,100))

    gen_imgs = generator.predict(noise)

    d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
    
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake) 

    noise = np.random.normal(0, 1, (batch_size, 100)) 
    valid_y = np.array([1] * batch_size)

    g_loss = combined.train_on_batch(noise, valid_y)

    print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

    if epoch % save_interval == 0:
      save_imgs(epoch)

In [ ]:
def save_imgs(epoch):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/mnist_%d.png" % epoch)
    plt.close()
#This function saves our images for us to view

In [ ]:
optimizer = Adam(0.0002, 0.5)

In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
__________________________________________________

In [ ]:
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=optimizer)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
dense_4 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dense_5 (Dense)              (None, 1024)             

In [ ]:
z = Input(shape=(100,))   #Our random input to the generator
img = generator(z)

In [ ]:
discriminator.trainable = False  

In [ ]:
valid = discriminator(img) 

In [ ]:
combined = Model(z, valid)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)


train(epochs=3000, batch_size=32, save_interval=10)

0 [D loss: 0.008919, acc.: 100.00%] [G loss: 4.360805]
1 [D loss: 0.018948, acc.: 100.00%] [G loss: 4.439970]
2 [D loss: 0.018085, acc.: 100.00%] [G loss: 4.560290]
3 [D loss: 0.010517, acc.: 100.00%] [G loss: 4.693010]
4 [D loss: 0.015157, acc.: 100.00%] [G loss: 4.452650]
5 [D loss: 0.010144, acc.: 100.00%] [G loss: 4.389310]
6 [D loss: 0.008026, acc.: 100.00%] [G loss: 4.336214]
7 [D loss: 0.017606, acc.: 100.00%] [G loss: 4.629028]
8 [D loss: 0.015454, acc.: 100.00%] [G loss: 4.609894]
9 [D loss: 0.038258, acc.: 100.00%] [G loss: 4.952139]
10 [D loss: 0.009246, acc.: 100.00%] [G loss: 4.869390]
11 [D loss: 0.019568, acc.: 100.00%] [G loss: 4.732875]
12 [D loss: 0.006501, acc.: 100.00%] [G loss: 4.439836]
13 [D loss: 0.022757, acc.: 100.00%] [G loss: 4.904056]
14 [D loss: 0.007912, acc.: 100.00%] [G loss: 4.855470]
15 [D loss: 0.023292, acc.: 100.00%] [G loss: 4.652661]
16 [D loss: 0.008556, acc.: 100.00%] [G loss: 4.783570]
17 [D loss: 0.016854, acc.: 100.00%] [G loss: 4.912864]
18